In [41]:
# will be using mnsit data set 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [42]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [43]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)
#xavier algorithm automatically determines the scale of initialization based on the number of input and output neurons.

In [44]:
#define tensorflow placeholders ad variables

In [45]:
#for generator 
Z = tf.placeholder(tf.float32, shape=[None, 100]) #input z dims 100 vector

G_W1 = tf.Variable(xavier_init([100, 128])) 
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784])) # last layer 786 dims vector which is 28x28 mnis image size

theta_G = [G_W1, G_W2, G_b1, G_b2] 

In [46]:
# For Discriminator
X = tf.placeholder(tf.float32, shape=[None, 784], name='X')  #input dims are 784 = 28x28 image

D_W1 = tf.Variable(xavier_init([784, 128]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[128]), name='D_b1')

D_W2 = tf.Variable(xavier_init([128, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2') 

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [47]:
def generator(z):
    G_h = tf.matmul(z, G_W1) + G_b1
    G_h1 = tf.nn.relu(G_h)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

def discriminator(x):
    D_h = tf.matmul(x, D_W1) + D_b1
    D_h1 = tf.nn.relu(D_h)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit


In [48]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X) #passing samples of real images
D_fake, D_logit_fake = discriminator(G_sample) #samples of Genrated image 

In [49]:
#Losses 
#type 1: using probability of image of being fake and real 
#D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
#G_loss = -tf.reduce_mean(tf.log(D_fake))

#type 2:
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

In [50]:
# Both loss will give same results, it's just a diff notation
#will use type 1 
#D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
#G_loss = -tf.reduce_mean(tf.log(D_fake))

In [51]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

In [52]:
mb_size = 128
Z_dim = 100
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz


In [53]:
if not os.path.exists('out/'):
    os.makedirs('out/')

In [54]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [55]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [56]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [57]:
i = 0
for it in range(1000000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.401
G_loss: 2.2

Iter: 1000
D loss: 0.001446
G_loss: 8.516

Iter: 2000
D loss: 0.01219
G_loss: 6.798

Iter: 3000
D loss: 0.02767
G_loss: 5.956

Iter: 4000
D loss: 0.06886
G_loss: 6.522

Iter: 5000
D loss: 0.1273
G_loss: 5.552

Iter: 6000
D loss: 0.2711
G_loss: 4.249

Iter: 7000
D loss: 0.2221
G_loss: 3.389

Iter: 8000
D loss: 0.2978
G_loss: 3.565

Iter: 9000
D loss: 0.5505
G_loss: 3.423

Iter: 10000
D loss: 0.4331
G_loss: 3.104

Iter: 11000
D loss: 0.372
G_loss: 3.12

Iter: 12000
D loss: 0.5654
G_loss: 2.462

Iter: 13000
D loss: 0.5051
G_loss: 2.688

Iter: 14000
D loss: 0.5271
G_loss: 2.807

Iter: 15000
D loss: 0.5909
G_loss: 2.62

Iter: 16000
D loss: 0.5471
G_loss: 2.426

Iter: 17000
D loss: 0.5555
G_loss: 2.428

Iter: 18000
D loss: 0.5859
G_loss: 2.116

Iter: 19000
D loss: 0.54
G_loss: 2.471

Iter: 20000
D loss: 0.6876
G_loss: 2.586

Iter: 21000
D loss: 0.6378
G_loss: 2.608

Iter: 22000
D loss: 0.6063
G_loss: 2.493

Iter: 23000
D loss: 0.6289
G_loss: 2.467

Iter: 24

Iter: 194000
D loss: 0.3657
G_loss: 2.898

Iter: 195000
D loss: 0.2593
G_loss: 3.151

Iter: 196000
D loss: 0.2505
G_loss: 2.845

Iter: 197000
D loss: 0.3319
G_loss: 3.104

Iter: 198000
D loss: 0.3556
G_loss: 2.476

Iter: 199000
D loss: 0.3762
G_loss: 2.723

Iter: 200000
D loss: 0.3719
G_loss: 2.932

Iter: 201000
D loss: 0.2046
G_loss: 3.381

Iter: 202000
D loss: 0.331
G_loss: 3.131

Iter: 203000
D loss: 0.3455
G_loss: 3.157

Iter: 204000
D loss: 0.2858
G_loss: 3.158

Iter: 205000
D loss: 0.3732
G_loss: 2.971

Iter: 206000
D loss: 0.3559
G_loss: 3.411

Iter: 207000
D loss: 0.4443
G_loss: 2.931

Iter: 208000
D loss: 0.1821
G_loss: 2.753

Iter: 209000
D loss: 0.2811
G_loss: 2.789

Iter: 210000
D loss: 0.3371
G_loss: 3.313

Iter: 211000
D loss: 0.3855
G_loss: 2.986

Iter: 212000
D loss: 0.3196
G_loss: 2.944

Iter: 213000
D loss: 0.3228
G_loss: 2.999

Iter: 214000
D loss: 0.2626
G_loss: 2.938

Iter: 215000
D loss: 0.2065
G_loss: 2.917

Iter: 216000
D loss: 0.3736
G_loss: 3.171

Iter: 217000

KeyboardInterrupt: 

In [58]:
# Stopping at Iter: 295000
# getting good results  